In [16]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [17]:
PATH = 'dataset/dataset_updated/training_set/'
PATH_VALIDATION = 'dataset/dataset_updated/validation_set/'
categories = ['drawings', 'engraving', 'iconography', 'painting', 'sculpture']

In [18]:
def extract_features(folder_path, dataset, label):
    sift = cv2.SIFT_create()
    orb = cv2.ORB_create()

    for image_name in os.listdir(folder_path):
        image_path = f"{folder_path}/{image_name}"
        image = cv2.imread(image_path)
        if image is None:
            continue
        resized_image = cv2.resize(image, (224, 224))
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

        sobelx = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
        sobel_edges = cv2.magnitude(sobelx, sobely).astype(np.uint8)

        sift_keypoints, sift_descriptors = sift.detectAndCompute(sobel_edges, None)
        orb_keypoints, orb_descriptors = orb.detectAndCompute(sobel_edges, None)

        if sift_descriptors is not None and orb_descriptors is not None:
            sift_mean = np.mean(sift_descriptors, axis=0)
            orb_mean = np.mean(orb_descriptors, axis=0)
            combined_features = np.hstack((sift_mean, orb_mean))
            dataset.append((combined_features, label, image))

In [19]:
train_features = []
val_features = []

for category in categories:
    folder_path = f"{PATH}/{category}"
    extract_features(folder_path, train_features, category)

for category in categories:
    folder_path = f"{PATH_VALIDATION}/{category}"
    extract_features(folder_path, val_features, category)

In [ ]:
X_train = [item[0] for item in train_features]
y_train = [item[1] for item in train_features]

X_val = [item[0] for item in val_features]
y_val = [item[1] for item in val_features]
images_val = [item[2] for item in val_features]

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)
clf = RandomForestClassifier(n_estimators=50000, random_state=42)
clf.fit(X_train, y_train_encoded)

y_pred = clf.predict(X_val)

print("Validation Classification Report:")
print(classification_report(y_val_encoded, y_pred, target_names=le.classes_))

accuracy = accuracy_score(y_val_encoded, y_pred)
print(f"Validation Accuracy: {accuracy:.2f}")

In [ ]:
import pickle

with open('model.pickle', 'wb') as file:
    pickle.dump(clf, file)
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(le, file)

In [ ]:
def display_all_validation_results(images, y_val, y_pred, le, items_per_page=10):
    total_images = len(images)
    pages = (total_images + items_per_page - 1) // items_per_page

    for page in range(pages):
        start_idx = page * items_per_page
        end_idx = min(start_idx + items_per_page, total_images)

        plt.figure(figsize=(15, 10))
        for i, idx in enumerate(range(start_idx, end_idx)):
            plt.subplot(2, items_per_page // 2, i + 1) 
            plt.imshow(cv2.cvtColor(images[idx], cv2.COLOR_BGR2RGB))
            plt.axis('off')
            true_label = le.inverse_transform([y_val[idx]])[0]
            predicted_label = le.inverse_transform([y_pred[idx]])[0]
            plt.title(f"True: {true_label}\nPred: {predicted_label}")
        plt.tight_layout()
        plt.show()

display_all_validation_results(images_val, y_val_encoded, y_pred, le, items_per_page=10)

In [22]:
def visualize_keypoint_matches(image1, image2):
    sift = cv2.SIFT_create()
    
    target1_grayscale = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    target1_object = cv2.GaussianBlur(target1_grayscale, (5, 5),0)
    
    target2_grayscale = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
    target2_object = cv2.GaussianBlur(target2_grayscale, (5, 5),0)

    kp1, des1 = sift.detectAndCompute(target1_object, None)
    des1 = np.float32(des1)
    kp2, des2 = sift.detectAndCompute(target2_object, None)
    des2 = np.float32(des2)

    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=10), dict(checks=100))
    matches = flann.knnMatch(des1, des2, k=2)  
    matchesmask = [[0, 0] for _ in range(0, len(matches))]  

    for i, (fm, sm) in enumerate(matches):
        if fm.distance < 0.7 * sm.distance:
            matchesmask[i] = [1, 0]
    
    best_matches_data = {  # Store the current best matching image and details
            'image_data': image2,
            'keypoint': kp2,
            'descriptor': des2,
            'match': matches,
            'matchesmask': matchesmask
        }
    
    result = cv2.drawMatchesKnn(
        image1,
        kp1,
        best_matches_data['image_data'],
        best_matches_data['keypoint'],
        best_matches_data['match'],
        None, 
        matchesMask = best_matches_data['matchesmask']
        )

    plt.figure(figsize=(12, 6))
    plt.imshow(result)
    plt.axis('off')
    plt.title("Keypoints Matching")
    plt.show()

In [23]:
sift = cv2.SIFT_create()
orb = cv2.ORB_create()

In [ ]:
target1 = cv2.imread('dataset/dataset_updated/local/drawings.jpeg')
target2 = cv2.imread('dataset/dataset_updated/local/drawings2.jpeg')

visualize_keypoint_matches(target1, target1)